In [7]:
import numpy as np
import shutil
import os
from tqdm import tqdm

In [8]:
import sys
sys.path.insert(1,"/usr/lib")
import gmsh


In [9]:
def writeCirclePts(r,N,z,charLen):
    # print("\n\n\n")
    cl_1 = []
    ptList=[]
    theta = np.linspace(0,2*np.pi,N)
    co = np.array([r*np.cos(theta),r*np.sin(theta),np.zeros(N)]).T
    for i,c in enumerate(co):
        ptList.append(gmsh.model.geo.add_point(c[0],c[1],z,charLen))
        # print(f"Point({ptList[-1]}) = "+"{"+f"{c[0]},{c[1]},{z},{charLen}"+"};")

    for i,tag in enumerate(ptList):
        if(i >= 1):
            # print(ptList[i-1],ptList[i])
            # print("// circle lines")
            cl_1.append(gmsh.model.geo.add_line(ptList[i-1],ptList[i]))
            # print(f"Line({cl_1[-1]}) = "+"{"+f"{ptList[i-1]},{ptList[i]}"+"};")
    # print("\n\n\n")
    return cl_1 , ptList

def writeWall(R,N,zb,zt,charLen):
    curve_loops = []
    planes = []
    theta = np.linspace(0,2*np.pi,N)
    z1 = zb
    z2 = zt
    r = R
    co = np.array([r*np.cos(theta),r*np.sin(theta),np.zeros(N)]).T

    for i in range(0,co.shape[0]-1):
        t1 = gmsh.model.geo.add_point(co[i,0],co[i,1],z1,charLen)
        t2 = gmsh.model.geo.add_point(co[i,0],co[i,1],z2,charLen)
        t3 = gmsh.model.geo.add_point(co[i+1,0],co[i+1,1],z2,charLen)
        t4 = gmsh.model.geo.add_point(co[i+1,0],co[i+1,1],z1,charLen)
        l1 = gmsh.model.geo.add_line(t1,t2)
        l2 = gmsh.model.geo.add_line(t2,t3)
        l3 = gmsh.model.geo.add_line(t3,t4)
        l4 = gmsh.model.geo.add_line(t4,t1)
        # print(f"Point({t1}) = "+"{"+f"{co[i,0]},{co[i,1]},{z1},{charLen}"+"};")
        # print(f"Point({t2}) = "+"{"+f"{co[i,0]},{co[i,1]},{z2},{charLen}"+"};")
        # print(f"Point({t3}) = "+"{"+f"{co[i+1,0]},{co[i+1,1]},{z2},{charLen}"+"};")
        # print(f"Point({t4}) = "+"{"+f"{co[i+1,0]},{co[i+1,1]},{z1},{charLen}"+"};")
        # print(f"Line({l1}) = "+"{"+f"{t1},{t2}"+"};")
        # print(f"Line({l2}) = "+"{"+f"{t2},{t3}"+"};")
        # print(f"Line({l3}) = "+"{"+f"{t3},{t4}"+"};")
        # print(f"Line({l4}) = "+"{"+f"{t4},{t1}"+"};")
        curve_loops.append(gmsh.model.geo.add_curve_loop([l1,l2,l3,l4]))
        # print(f"Curve Loop({curve_loops[-1]}) = "+"{"+f"{l1},{l2},{l3},{l4}"+"};")
        planes.append(gmsh.model.geo.add_surface_filling([curve_loops[-1]]))
        # print(f"Plane Surface({planes[-1]}) = "+"{"+f"{curve_loops[-1]}"+"};")
    return curve_loops, planes

In [10]:
charLen = 1e-7
N = 400
# R = 15e-6
# W = 4e-6
z_off = 0.2e-6 # thickness
num_divs = 3
charLen_side = 1e-7

Rs = np.linspace(10e-6,15e-6,10)
Ws = np.linspace(7e-6,3e-6,10)

### Mesh volume

In [ ]:
count = 0
for R,W in tqdm(zip(Rs,Ws)):
    print(f"start with iteration {count}")
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity",0)
    gmsh.model.add("t")


    cl_1, pts_1 = writeCirclePts(R,N,-z_off/2,charLen)
    cl_2, pts_2 = writeCirclePts(R-W,N,-z_off/2,charLen)
    cl_3, pts_3 = writeCirclePts(R,N,z_off/2,charLen)
    cl_4, pts_4  = writeCirclePts(R-W,N, z_off/2,charLen)

    #side walls (outer)

    # writeWall(R,N,-z_off/2,z_off/2,charLen)
    # writeWall(R-W,N,-z_off/2,z_off/2,charLen)

    outer_surfaces = []
    inner_surfaces = []

    z_inc = np.linspace(-z_off/2,z_off/2,num_divs)
    for i in range(0,z_inc.shape[0]-1):
        _, outer_planes = writeWall(R,N,z_inc[i],z_inc[i+1],charLen_side)
        _, inner_planes = writeWall(R-W,N,z_inc[i],z_inc[i+1],charLen_side)
        outer_surfaces.extend(outer_planes)
        inner_surfaces.extend(inner_planes)

    # top and bottom loops
    loop2 = gmsh.model.geo.add_curve_loop(cl_1)
    val = f"Curve Loop({loop2}) = "+"{"
    for c in cl_1:
        val += f"{c},"
    val = val[0:-1] + "};"
    # print(val)

    loop3 = gmsh.model.geo.add_curve_loop(cl_2)


    val = f"Curve Loop({loop3}) = "+"{"
    for c in cl_2:
        val += f"{c},"
    val = val[0:-1] + "};"
    # print(val)

    p1 = gmsh.model.geo.add_plane_surface([loop2,loop3])

    # print(f"Plane Surface({p1}) = "+"{"+f"{loop2},{loop3}"+"};")

    loop4 = gmsh.model.geo.add_curve_loop(cl_3)

    val = f"Curve Loop({loop4}) = "+"{"
    for c in cl_3:
        val += f"{c},"
    val = val[0:-1] + "};"
    # print(val)

    loop5 = gmsh.model.geo.add_curve_loop(cl_4)

    val = f"Curve Loop({loop5}) = "+"{"
    for c in cl_4:
        val += f"{c},"
    val = val[0:-1] + "};"
    # print(val)

    p2 = gmsh.model.geo.add_plane_surface([loop4,loop5])

    # print(f"Plane Surface({p2}) = "+"{"+f"{loop4},{loop5}"+"};")

    # gmsh.model.geo.add_volume()

    val = []
    val.append(p1)
    val.extend(outer_surfaces)
    val.append(p2)
    val.extend(inner_surfaces)
    surf_loop = gmsh.model.geo.add_surface_loop(val)

    vol = gmsh.model.geo.add_volume([surf_loop])

    pt1 = gmsh.model.geo.add_point(0,0,-1e-6,1.5)
    pt2 = gmsh.model.geo.add_point(0,0,1e-6,1.5)
    fone = gmsh.model.geo.add_line(pt1,pt2)

    # gmsh.model.addPhysicalGroup(2,val,name="Planes")

    gmsh.model.geo.removeAllDuplicates()

    gmsh.model.geo.add_physical_group(3,[vol],name="T0_ybco")
    gmsh.model.geo.add_physical_group(1,[fone],name="fone")
    gmsh.model.geo.synchronize()
    # gmsh.fltk.run()
    gmsh.option.setNumber("Mesh.MshFileVersion",2.2)
    gmsh.option.setNumber("Mesh.Algorithm",5)
    gmsh.option.setNumber("Geometry.Points",0)
    gmsh.option.setNumber("Geometry.Curves",0)
    gmsh.option.setNumber("Mesh.Algorithm3D",1)

    gmsh.model.mesh.generate(3)
    gmsh.model.mesh.removeDuplicateElements()
    
    folder_name = f"test{count}"
    if os.path.exists(folder_name) and os.path.isdir(folder_name):
        # If the folder exists, delete it
        shutil.rmtree(folder_name)
    os.mkdir(folder_name)
    os.system("cp settings_washer.json ./test{count}")
    
    os.chdir(f"./test{count}")
    gmsh.write(f"test.msh")
    # gmsh.fltk.run()
    gmsh.finalize()
    print(f"done with iteration {count}")
    count = count + 1
    os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/SquareThinFilmWire/TestSetup")




In [15]:
count = 10
R = 18e-6
W = 1e-6
print(f"start with iteration {count}")
gmsh.initialize()
gmsh.option.setNumber("General.Verbosity",0)
gmsh.model.add("t")
cl_1, pts_1 = writeCirclePts(R,N,-z_off/2,charLen)
cl_2, pts_2 = writeCirclePts(R-W,N,-z_off/2,charLen)
cl_3, pts_3 = writeCirclePts(R,N,z_off/2,charLen)
cl_4, pts_4  = writeCirclePts(R-W,N, z_off/2,charLen)
#side walls (outer)
# writeWall(R,N,-z_off/2,z_off/2,charLen)
# writeWall(R-W,N,-z_off/2,z_off/2,charLen)
outer_surfaces = []
inner_surfaces = []
z_inc = np.linspace(-z_off/2,z_off/2,num_divs)
for i in range(0,z_inc.shape[0]-1):
    _, outer_planes = writeWall(R,N,z_inc[i],z_inc[i+1],charLen_side)
    _, inner_planes = writeWall(R-W,N,z_inc[i],z_inc[i+1],charLen_side)
    outer_surfaces.extend(outer_planes)
    inner_surfaces.extend(inner_planes)
# top and bottom loops
loop2 = gmsh.model.geo.add_curve_loop(cl_1)
val = f"Curve Loop({loop2}) = "+"{"
for c in cl_1:
    val += f"{c},"
val = val[0:-1] + "};"
# print(val)
loop3 = gmsh.model.geo.add_curve_loop(cl_2)
val = f"Curve Loop({loop3}) = "+"{"
for c in cl_2:
    val += f"{c},"
val = val[0:-1] + "};"
# print(val)
p1 = gmsh.model.geo.add_plane_surface([loop2,loop3])
# print(f"Plane Surface({p1}) = "+"{"+f"{loop2},{loop3}"+"};")
loop4 = gmsh.model.geo.add_curve_loop(cl_3)
val = f"Curve Loop({loop4}) = "+"{"
for c in cl_3:
    val += f"{c},"
val = val[0:-1] + "};"
# print(val)
loop5 = gmsh.model.geo.add_curve_loop(cl_4)
val = f"Curve Loop({loop5}) = "+"{"
for c in cl_4:
    val += f"{c},"
val = val[0:-1] + "};"
# print(val)
p2 = gmsh.model.geo.add_plane_surface([loop4,loop5])
# print(f"Plane Surface({p2}) = "+"{"+f"{loop4},{loop5}"+"};")
# gmsh.model.geo.add_volume()
val = []
val.append(p1)
val.extend(outer_surfaces)
val.append(p2)
val.extend(inner_surfaces)
surf_loop = gmsh.model.geo.add_surface_loop(val)
vol = gmsh.model.geo.add_volume([surf_loop])
pt1 = gmsh.model.geo.add_point(0,0,-1e-6,1.5)
pt2 = gmsh.model.geo.add_point(0,0,1e-6,1.5)
fone = gmsh.model.geo.add_line(pt1,pt2)
# gmsh.model.addPhysicalGroup(2,val,name="Planes")
gmsh.model.geo.removeAllDuplicates()
gmsh.model.geo.add_physical_group(3,[vol],name="T0_ybco")
gmsh.model.geo.add_physical_group(1,[fone],name="fone")
gmsh.model.geo.synchronize()

gmsh.option.setNumber("Mesh.MshFileVersion",2.2)
gmsh.option.setNumber("Mesh.Algorithm",5)
gmsh.option.setNumber("Geometry.Points",0)
gmsh.option.setNumber("Geometry.Curves",0)
gmsh.option.setNumber("Mesh.Algorithm3D",1)



gmsh.model.mesh.generate(3)
gmsh.model.mesh.removeDuplicateElements()

folder_name = f"test{count}"
if os.path.exists(folder_name) and os.path.isdir(folder_name):
    # If the folder exists, delete it
    shutil.rmtree(folder_name)
os.mkdir(folder_name)
os.system(f"cp settings_washer.json ./test{count}")

os.chdir(f"./test{count}")


gmsh.write(f"test.msh")

gmsh.finalize()
print(f"done with iteration {count}")
count = count + 1
os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/SquareThinFilmWire/TestSetup")

start with iteration 10
done with iteration 10


In [ ]:
charLen = 3e-7
N = 400
# R = 15e-6
# W = 4e-6
z_off = 0.2e-6 # thickness
num_divs = 15
charLen_side = 0.4e-7

Rs = np.linspace(10e-6,15e-6,10)
Ws = np.linspace(7e-6,3e-6,10)

### Surface

In [ ]:
count = 0
for R,W in zip(Rs,Ws):
    print(f"Start with iteration {count}")
    gmsh.initialize()

    gmsh.model.add("t")


    cl_1, pts_1 = writeCirclePts(R,N,-z_off/2,charLen)
    cl_2, pts_2 = writeCirclePts(R-W,N,-z_off/2,charLen)
    cl_3, pts_3 = writeCirclePts(R,N,z_off/2,charLen)
    cl_4, pts_4  = writeCirclePts(R-W,N, z_off/2,charLen)

    #side walls (outer)

    # writeWall(R,N,-z_off/2,z_off/2,charLen)
    # writeWall(R-W,N,-z_off/2,z_off/2,charLen)

    outer_surfaces = []
    inner_surfaces = []
    z_inc = np.linspace(-z_off/2,z_off/2,num_divs)
    for i in range(0,z_inc.shape[0]-1):
        _, outer_planes = writeWall(R,N,z_inc[i],z_inc[i+1],charLen_side)
        _, inner_planes = writeWall(R-W,N,z_inc[i],z_inc[i+1],charLen_side)
        outer_surfaces.extend(outer_planes)
        inner_surfaces.extend(inner_planes)

    # top and bottom loops
    loop2 = gmsh.model.geo.add_curve_loop(cl_1)
    val = f"Curve Loop({loop2}) = "+"{"
    for c in cl_1:
        val += f"{c},"
    val = val[0:-1] + "};"
    # print(val)

    loop3 = gmsh.model.geo.add_curve_loop(cl_2)


    val = f"Curve Loop({loop3}) = "+"{"
    for c in cl_2:
        val += f"{c},"
    val = val[0:-1] + "};"
    # print(val)

    p1 = gmsh.model.geo.add_plane_surface([loop2,loop3])

    # print(f"Plane Surface({p1}) = "+"{"+f"{loop2},{loop3}"+"};")

    loop4 = gmsh.model.geo.add_curve_loop(cl_3)

    val = f"Curve Loop({loop4}) = "+"{"
    for c in cl_3:
        val += f"{c},"
    val = val[0:-1] + "};"
    # print(val)

    loop5 = gmsh.model.geo.add_curve_loop(cl_4)

    val = f"Curve Loop({loop5}) = "+"{"
    for c in cl_4:
        val += f"{c},"
    val = val[0:-1] + "};"
    # print(val)

    p2 = gmsh.model.geo.add_plane_surface([loop4,loop5])

    # print(f"Plane Surface({p2}) = "+"{"+f"{loop4},{loop5}"+"};")

    # gmsh.model.geo.add_volume()

    val = []
    val.append(p1)
    val.extend(outer_surfaces)
    val.append(p2)
    val.extend(inner_surfaces)

    # surf_loop = gmsh.model.geo.add_surface_loop(val)


    gmsh.model.geo.removeAllDuplicates()

    gmsh.model.geo.add_physical_group(2,val,name="FieldPlane0")

    gmsh.model.geo.synchronize()
    # gmsh.fltk.run()
    gmsh.option.setNumber("Mesh.MshFileVersion",2.2)
    gmsh.option.setNumber("Mesh.Algorithm",5)
    gmsh.option.setNumber("Geometry.Points",0)
    gmsh.option.setNumber("Geometry.Curves",0)
    gmsh.option.setNumber("Mesh.Algorithm3D",1)

    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.removeDuplicateElements()
    
    os.chdir(f"./test{count}")
    gmsh.write("testFIELD.msh")
    # gmsh.fltk.run()
    gmsh.finalize()
    print(f"Done with iteration {count}")
    count = count + 1
    os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/SquareThinFilmWire/TestSetup")

In [16]:
charLen = 3e-7
N = 400
R = 15e-6
W = 4e-6
z_off = 0.2e-6 # thickness
num_divs = 15
charLen_side = 0.4e-7
count = 10
R = 18e-6
W = 1e-6

print(f"Start with iteration {count}")
gmsh.initialize()
gmsh.model.add("t")

cl_1, pts_1 = writeCirclePts(R,N,-z_off/2,charLen)
cl_2, pts_2 = writeCirclePts(R-W,N,-z_off/2,charLen)
cl_3, pts_3 = writeCirclePts(R,N,z_off/2,charLen)
cl_4, pts_4  = writeCirclePts(R-W,N, z_off/2,charLen)
#side walls (outer)
# writeWall(R,N,-z_off/2,z_off/2,charLen)
# writeWall(R-W,N,-z_off/2,z_off/2,charLen)
outer_surfaces = []
inner_surfaces = []
z_inc = np.linspace(-z_off/2,z_off/2,num_divs)
for i in range(0,z_inc.shape[0]-1):
    _, outer_planes = writeWall(R,N,z_inc[i],z_inc[i+1],charLen_side)
    _, inner_planes = writeWall(R-W,N,z_inc[i],z_inc[i+1],charLen_side)
    outer_surfaces.extend(outer_planes)
    inner_surfaces.extend(inner_planes)
# top and bottom loops
loop2 = gmsh.model.geo.add_curve_loop(cl_1)
val = f"Curve Loop({loop2}) = "+"{"
for c in cl_1:
    val += f"{c},"
val = val[0:-1] + "};"
# print(val)
loop3 = gmsh.model.geo.add_curve_loop(cl_2)
val = f"Curve Loop({loop3}) = "+"{"
for c in cl_2:
    val += f"{c},"
val = val[0:-1] + "};"
# print(val)
p1 = gmsh.model.geo.add_plane_surface([loop2,loop3])
# print(f"Plane Surface({p1}) = "+"{"+f"{loop2},{loop3}"+"};")
loop4 = gmsh.model.geo.add_curve_loop(cl_3)
val = f"Curve Loop({loop4}) = "+"{"
for c in cl_3:
    val += f"{c},"
val = val[0:-1] + "};"
# print(val)
loop5 = gmsh.model.geo.add_curve_loop(cl_4)
val = f"Curve Loop({loop5}) = "+"{"
for c in cl_4:
    val += f"{c},"
val = val[0:-1] + "};"
# print(val)
p2 = gmsh.model.geo.add_plane_surface([loop4,loop5])
# print(f"Plane Surface({p2}) = "+"{"+f"{loop4},{loop5}"+"};")
# gmsh.model.geo.add_volume()
val = []
val.append(p1)
val.extend(outer_surfaces)
val.append(p2)
val.extend(inner_surfaces)
# surf_loop = gmsh.model.geo.add_surface_loop(val)
gmsh.model.geo.removeAllDuplicates()
gmsh.model.geo.add_physical_group(2,val,name="FieldPlane0")
gmsh.model.geo.synchronize()
# gmsh.fltk.run()
gmsh.option.setNumber("Mesh.MshFileVersion",2.2)
gmsh.option.setNumber("Mesh.Algorithm",5)
gmsh.option.setNumber("Geometry.Points",0)
gmsh.option.setNumber("Geometry.Curves",0)
gmsh.option.setNumber("Mesh.Algorithm3D",1)
gmsh.model.mesh.generate(2)
gmsh.model.mesh.removeDuplicateElements()

os.chdir(f"./test{count}")
gmsh.write("testFIELD.msh")
# gmsh.fltk.run()
gmsh.finalize()
print(f"Done with iteration {count}")
count = count + 1
os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/SquareThinFilmWire/TestSetup")

Start with iteration 10
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] M